In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
df=pd.read_csv("car_resale_prices.csv")

In [4]:
df

,Unnamed: 0,full_name,resale_price,registered_year,engine_capacity,insurance,transmission_type,kms_driven,owner_type,fuel_type,max_power,seats,mileage,body_type,city
0,0,2017 Maruti Baleno 1.2 Alpha,₹ 5.45 Lakh,2017,1197 cc,Third Party insurance,Manual,"40,000 Kms",First Owner,Petrol,83.1bhp,5.0,21.4 kmpl,Hatchback,Agra
1,1,2018 Tata Hexa XTA,₹ 10 Lakh,2018,2179 cc,Third Party insurance,Automatic,"70,000 Kms",First Owner,Diesel,153.86bhp,7.0,17.6 kmpl,MUV,Agra
2,2,2015 Maruti Swift Dzire VXI,₹ 4.50 Lakh,2015,1197 cc,Third Party insurance,Manual,"70,000 Kms",Second Owner,Petrol,83.14bhp,5.0,20.85 kmpl,Sedan,Agra
3,3,2015 Maruti Swift Dzire VXI,₹ 4.50 Lakh,2015,1197 cc,Third Party insurance,Manual,"70,000 Kms",Second Owner,Petrol,83.14bhp,5.0,20.85 kmpl,Sedan,Agra
4,4,2009 Hyundai i10 Magna 1.1,₹ 1.60 Lakh,2009,1086 cc,Third Party insurance,Manual,"80,000 Kms",First Owner,Petrol,68.05bhp,5.0,19.81 kmpl,Hatchback,Agra
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17441,17441,2013 Honda Amaze VX i-Vtech,₹ 3.25 Lakh,Jul 2013,1198 cc,Comprehensive,Manual,"89,000 Kms",Second Owner,Petrol,86.7bhp,5.0,18 kmpl,Sedan,Delhi
17442,17442,2016 Toyota Camry 2.5 Hybrid,₹ 20.75 Lakh,Jun 2016,2494 cc,Comprehensive,Automatic,"68,000 Kms",First Owner,Petrol,157.7bhp,5.0,19.16 kmpl,Sedan,Delhi
17443,17443,2016 Toyota Corolla Altis GL MT,₹ 8.35 Lakh,Jun 2016,1798 cc,Comprehensive,Manual,"81,000 Kms",First Owner,Petrol,138.03bhp,5.0,14.28 kmpl,Sedan,Delhi
17444,17444,2019 Hyundai Creta 1.6 CRDi AT SX Plus,₹ 13.95 Lakh,Jun 2019,1582 cc,Comprehensive,Automatic,"20,000 Kms",First Owner,Diesel,126.2bhp,5.0,17.01 kmpl,SUV,Delhi


In [5]:
df.describe()

,Unnamed: 0,seats
count,17446.000000,17436.000000
mean,8722.500000,5.205494
std,5036.370734,0.669321
min,0.000000,2.000000
25%,4361.250000,5.000000
50%,8722.500000,5.000000
75%,13083.750000,5.000000
max,17445.000000,14.000000


In [6]:
# --- Check for Null Values in Each Column ---
print("Null values in each column:")
for column in df.columns:
    null_count = df[column].isnull().sum()
    if null_count > 0:
        print(f"- {column}: {null_count} null values")
    else:
        print(f"- {column}: No null values")

Null values in each column:
- Unnamed: 0: No null values
- full_name: No null values
- resale_price: No null values
- registered_year: 69 null values
- engine_capacity: 14 null values
- insurance: 7 null values
- transmission_type: No null values
- kms_driven: 3 null values
- owner_type: 45 null values
- fuel_type: No null values
- max_power: 102 null values
- seats: 10 null values
- mileage: 508 null values
- body_type: No null values
- city: No null values


In [7]:
# Clean 'engine_capacity' by removing "cc" and converting to numeric
df['engine_capacity'] = df['engine_capacity'].str.replace('cc', '').str.strip()
df['engine_capacity'] = pd.to_numeric(df['engine_capacity'], errors='coerce')

# Clean 'max_power' by removing "bhp" and converting to numeric
df['max_power'] = df['max_power'].str.replace('bhp', '').str.strip()
df['max_power'] = pd.to_numeric(df['max_power'], errors='coerce')

print("Data types after cleaning:")
print(df[['engine_capacity', 'max_power']].info())

Data types after cleaning:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17446 entries, 0 to 17445
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   engine_capacity  17432 non-null  float64
 1   max_power        16924 non-null  float64
dtypes: float64(2)
memory usage: 272.7 KB
None


In [8]:
from sklearn.linear_model import LinearRegression

# Create a temporary DataFrame with no nulls in either column to train our model
train_df = df.dropna(subset=['engine_capacity', 'max_power'])

# --- Model 1: Predict max_power from engine_capacity ---
power_model = LinearRegression()
X_train_power = train_df[['engine_capacity']]
y_train_power = train_df['max_power']
power_model.fit(X_train_power, y_train_power)

# Identify rows where max_power is null but engine_capacity is not
impute_power_rows = df['max_power'].isnull() & df['engine_capacity'].notnull()

# Predict and fill
df.loc[impute_power_rows, 'max_power'] = power_model.predict(df.loc[impute_power_rows, ['engine_capacity']])


# --- Model 2: Predict engine_capacity from max_power ---
engine_model = LinearRegression()
X_train_engine = train_df[['max_power']]
y_train_engine = train_df['engine_capacity']
engine_model.fit(X_train_engine, y_train_engine)

# Identify rows where engine_capacity is null but max_power is not
impute_engine_rows = df['engine_capacity'].isnull() & df['max_power'].notnull()

# Predict and fill
df.loc[impute_engine_rows, 'engine_capacity'] = engine_model.predict(df.loc[impute_engine_rows, ['max_power']])

print("\nImputation using regression complete.")


Imputation using regression complete.


In [9]:
for column in df.columns:
    null_count = df[column].isnull().sum()
    if null_count > 0:
        print(f"- {column}: {null_count} null values")
    else:
        print(f"- {column}: No null values")

- Unnamed: 0: No null values
- full_name: No null values
- resale_price: No null values
- registered_year: 69 null values
- engine_capacity: 5 null values
- insurance: 7 null values
- transmission_type: No null values
- kms_driven: 3 null values
- owner_type: 45 null values
- fuel_type: No null values
- max_power: 5 null values
- seats: 10 null values
- mileage: 508 null values
- body_type: No null values
- city: No null values


In [10]:
mileage_numeric = pd.to_numeric(df['mileage'].str.split(' ').str[0], errors='coerce')
average_mileage = mileage_numeric.mean()
print(f"The average mileage is: {average_mileage:.2f} km/L (or km/kg)")

The average mileage is: 19.40 km/L (or km/kg)


In [11]:
# First, clean the mileage column and convert it to numeric, as we did before.
# We'll store this in the DataFrame itself now.
df['mileage_numeric'] = pd.to_numeric(df['mileage'].str.split(' ').str[0], errors='coerce')
# The corrected, future-proof way
fill_values = df.groupby('fuel_type')['mileage_numeric'].transform('mean')
df['mileage_numeric'] = df['mileage_numeric'].fillna(fill_values)

print("Null values in 'mileage_numeric' after group-based imputation:")
print(df['mileage_numeric'].isnull().sum())

Null values in 'mileage_numeric' after group-based imputation:
0


In [12]:
for column in df.columns:
    null_count = df[column].isnull().sum()
    if null_count > 0:
        print(f"- {column}: {null_count} null values")
    else:
        print(f"- {column}: No null values")

- Unnamed: 0: No null values
- full_name: No null values
- resale_price: No null values
- registered_year: 69 null values
- engine_capacity: 5 null values
- insurance: 7 null values
- transmission_type: No null values
- kms_driven: 3 null values
- owner_type: 45 null values
- fuel_type: No null values
- max_power: 5 null values
- seats: 10 null values
- mileage: 508 null values
- body_type: No null values
- city: No null values
- mileage_numeric: No null values


In [13]:
df.drop(columns=["mileage"],inplace=True)

In [14]:
df

,Unnamed: 0,full_name,resale_price,registered_year,engine_capacity,insurance,transmission_type,kms_driven,owner_type,fuel_type,max_power,seats,body_type,city,mileage_numeric
0,0,2017 Maruti Baleno 1.2 Alpha,₹ 5.45 Lakh,2017,1197.0,Third Party insurance,Manual,"40,000 Kms",First Owner,Petrol,83.10,5.0,Hatchback,Agra,21.40
1,1,2018 Tata Hexa XTA,₹ 10 Lakh,2018,2179.0,Third Party insurance,Automatic,"70,000 Kms",First Owner,Diesel,153.86,7.0,MUV,Agra,17.60
2,2,2015 Maruti Swift Dzire VXI,₹ 4.50 Lakh,2015,1197.0,Third Party insurance,Manual,"70,000 Kms",Second Owner,Petrol,83.14,5.0,Sedan,Agra,20.85
3,3,2015 Maruti Swift Dzire VXI,₹ 4.50 Lakh,2015,1197.0,Third Party insurance,Manual,"70,000 Kms",Second Owner,Petrol,83.14,5.0,Sedan,Agra,20.85
4,4,2009 Hyundai i10 Magna 1.1,₹ 1.60 Lakh,2009,1086.0,Third Party insurance,Manual,"80,000 Kms",First Owner,Petrol,68.05,5.0,Hatchback,Agra,19.81
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17441,17441,2013 Honda Amaze VX i-Vtech,₹ 3.25 Lakh,Jul 2013,1198.0,Comprehensive,Manual,"89,000 Kms",Second Owner,Petrol,86.70,5.0,Sedan,Delhi,18.00
17442,17442,2016 Toyota Camry 2.5 Hybrid,₹ 20.75 Lakh,Jun 2016,2494.0,Comprehensive,Automatic,"68,000 Kms",First Owner,Petrol,157.70,5.0,Sedan,Delhi,19.16
17443,17443,2016 Toyota Corolla Altis GL MT,₹ 8.35 Lakh,Jun 2016,1798.0,Comprehensive,Manual,"81,000 Kms",First Owner,Petrol,138.03,5.0,Sedan,Delhi,14.28
17444,17444,2019 Hyundai Creta 1.6 CRDi AT SX Plus,₹ 13.95 Lakh,Jun 2019,1582.0,Comprehensive,Automatic,"20,000 Kms",First Owner,Diesel,126.20,5.0,SUV,Delhi,17.01


In [15]:
df['kms_driven'] = df['kms_driven'].str.replace('Kms', ' ', regex=False)
df['kms_driven'] = df['kms_driven'].str.replace(',', '', regex=False)
df['kms_driven'] = df['kms_driven'].str.strip()
df['kms_driven'] = pd.to_numeric(df['kms_driven'], errors='coerce')

In [16]:
df

,Unnamed: 0,full_name,resale_price,registered_year,engine_capacity,insurance,transmission_type,kms_driven,owner_type,fuel_type,max_power,seats,body_type,city,mileage_numeric
0,0,2017 Maruti Baleno 1.2 Alpha,₹ 5.45 Lakh,2017,1197.0,Third Party insurance,Manual,40000.0,First Owner,Petrol,83.10,5.0,Hatchback,Agra,21.40
1,1,2018 Tata Hexa XTA,₹ 10 Lakh,2018,2179.0,Third Party insurance,Automatic,70000.0,First Owner,Diesel,153.86,7.0,MUV,Agra,17.60
2,2,2015 Maruti Swift Dzire VXI,₹ 4.50 Lakh,2015,1197.0,Third Party insurance,Manual,70000.0,Second Owner,Petrol,83.14,5.0,Sedan,Agra,20.85
3,3,2015 Maruti Swift Dzire VXI,₹ 4.50 Lakh,2015,1197.0,Third Party insurance,Manual,70000.0,Second Owner,Petrol,83.14,5.0,Sedan,Agra,20.85
4,4,2009 Hyundai i10 Magna 1.1,₹ 1.60 Lakh,2009,1086.0,Third Party insurance,Manual,80000.0,First Owner,Petrol,68.05,5.0,Hatchback,Agra,19.81
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17441,17441,2013 Honda Amaze VX i-Vtech,₹ 3.25 Lakh,Jul 2013,1198.0,Comprehensive,Manual,89000.0,Second Owner,Petrol,86.70,5.0,Sedan,Delhi,18.00
17442,17442,2016 Toyota Camry 2.5 Hybrid,₹ 20.75 Lakh,Jun 2016,2494.0,Comprehensive,Automatic,68000.0,First Owner,Petrol,157.70,5.0,Sedan,Delhi,19.16
17443,17443,2016 Toyota Corolla Altis GL MT,₹ 8.35 Lakh,Jun 2016,1798.0,Comprehensive,Manual,81000.0,First Owner,Petrol,138.03,5.0,Sedan,Delhi,14.28
17444,17444,2019 Hyundai Creta 1.6 CRDi AT SX Plus,₹ 13.95 Lakh,Jun 2019,1582.0,Comprehensive,Automatic,20000.0,First Owner,Diesel,126.20,5.0,SUV,Delhi,17.01


In [17]:
# List the columns you intend to process
numerical_cols = ['registered_year', 'engine_capacity', 'kms_driven', 'max_power', 'seats']
categorical_cols = ['insurance', 'owner_type']

# --- Step 1: Force all numerical columns to a numeric type ---
# This is a robust way to prevent the TypeError
print("Ensuring all target columns are numeric...")
for col in numerical_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# --- Step 2: Now, fill the null values ---
print("Filling null values...")

# Fill Numerical Columns with the Mean
for col in numerical_cols:
    mean_value = df[col].mean()
    df[col] = df[col].fillna(mean_value)

    # For columns that should be integers, round the values
    if col in ['registered_year', 'seats']:
        df[col] = df[col].round().astype(int)

# Fill Categorical Columns with the Mode
for col in categorical_cols:
    mode_value = df[col].mode()[0]
    df[col] = df[col].fillna(mode_value)

print("\nImputation complete.")

# --- Step 3: Final Verification ---
print("\nFinal null value check:")
print(df.isnull().sum())

Ensuring all target columns are numeric...
Filling null values...

Imputation complete.

Final null value check:
Unnamed: 0           0
full_name            0
resale_price         0
registered_year      0
engine_capacity      0
insurance            0
transmission_type    0
kms_driven           0
owner_type           0
fuel_type            0
max_power            0
seats                0
body_type            0
city                 0
mileage_numeric      0
dtype: int64


In [18]:
duplicate_count = df.duplicated().sum()
print(f"Found {duplicate_count} duplicate rows.")

Found 0 duplicate rows.


In [19]:
df

,Unnamed: 0,full_name,resale_price,registered_year,engine_capacity,insurance,transmission_type,kms_driven,owner_type,fuel_type,max_power,seats,body_type,city,mileage_numeric
0,0,2017 Maruti Baleno 1.2 Alpha,₹ 5.45 Lakh,2017,1197.0,Third Party insurance,Manual,40000.0,First Owner,Petrol,83.10,5,Hatchback,Agra,21.40
1,1,2018 Tata Hexa XTA,₹ 10 Lakh,2018,2179.0,Third Party insurance,Automatic,70000.0,First Owner,Diesel,153.86,7,MUV,Agra,17.60
2,2,2015 Maruti Swift Dzire VXI,₹ 4.50 Lakh,2015,1197.0,Third Party insurance,Manual,70000.0,Second Owner,Petrol,83.14,5,Sedan,Agra,20.85
3,3,2015 Maruti Swift Dzire VXI,₹ 4.50 Lakh,2015,1197.0,Third Party insurance,Manual,70000.0,Second Owner,Petrol,83.14,5,Sedan,Agra,20.85
4,4,2009 Hyundai i10 Magna 1.1,₹ 1.60 Lakh,2009,1086.0,Third Party insurance,Manual,80000.0,First Owner,Petrol,68.05,5,Hatchback,Agra,19.81
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17441,17441,2013 Honda Amaze VX i-Vtech,₹ 3.25 Lakh,2016,1198.0,Comprehensive,Manual,89000.0,Second Owner,Petrol,86.70,5,Sedan,Delhi,18.00
17442,17442,2016 Toyota Camry 2.5 Hybrid,₹ 20.75 Lakh,2016,2494.0,Comprehensive,Automatic,68000.0,First Owner,Petrol,157.70,5,Sedan,Delhi,19.16
17443,17443,2016 Toyota Corolla Altis GL MT,₹ 8.35 Lakh,2016,1798.0,Comprehensive,Manual,81000.0,First Owner,Petrol,138.03,5,Sedan,Delhi,14.28
17444,17444,2019 Hyundai Creta 1.6 CRDi AT SX Plus,₹ 13.95 Lakh,2016,1582.0,Comprehensive,Automatic,20000.0,First Owner,Diesel,126.20,5,SUV,Delhi,17.01


In [20]:
# Create the 'car_age' feature
current_year = 2025
df['car_age'] = current_year - df['registered_year']

# Now, drop the original 'registered_year' column
df.drop(columns=['registered_year'], inplace=True)

print("Created 'car_age' feature and dropped 'registered_year'.")
print(df[['car_age']].head())

Created 'car_age' feature and dropped 'registered_year'.
   car_age
0        8
1        7
2       10
3       10
4       16


In [21]:
# Ensure you have a dataframe with only numeric types
df_numeric = df.select_dtypes(include=['number'])

# Calculate the correlation matrix
correlation_matrix = df_numeric.corr()

# Unstack the matrix to create a list of all pairs
corr_pairs = correlation_matrix.unstack()

# Sort the pairs in descending order
sorted_pairs = corr_pairs.sort_values(kind="quicksort", ascending=False)

# Remove self-correlations (where a feature is correlated with itself, always 1.0)
strong_pairs = sorted_pairs[sorted_pairs != 1.0]

print("--- Top Correlated Feature Pairs ---")
print(strong_pairs.head(20))

--- Top Correlated Feature Pairs ---
engine_capacity  max_power          0.835000
max_power        engine_capacity    0.835000
seats            engine_capacity    0.428607
engine_capacity  seats              0.428607
max_power        seats              0.206193
seats            max_power          0.206193
car_age          kms_driven         0.171685
kms_driven       car_age            0.171685
engine_capacity  kms_driven         0.105688
kms_driven       engine_capacity    0.105688
max_power        Unnamed: 0         0.072335
Unnamed: 0       max_power          0.072335
engine_capacity  Unnamed: 0         0.070928
Unnamed: 0       engine_capacity    0.070928
kms_driven       seats              0.070915
seats            kms_driven         0.070915
engine_capacity  car_age            0.044267
car_age          engine_capacity    0.044267
seats            Unnamed: 0         0.029934
Unnamed: 0       seats              0.029934
dtype: float64


In [22]:
# Drop the 'max_power' column from the DataFrame
df.drop(columns=['max_power'], inplace=True)

print("Successfully removed the 'max_power' column.")

Successfully removed the 'max_power' column.


In [23]:
df

,Unnamed: 0,full_name,resale_price,engine_capacity,insurance,transmission_type,kms_driven,owner_type,fuel_type,seats,body_type,city,mileage_numeric,car_age
0,0,2017 Maruti Baleno 1.2 Alpha,₹ 5.45 Lakh,1197.0,Third Party insurance,Manual,40000.0,First Owner,Petrol,5,Hatchback,Agra,21.40,8
1,1,2018 Tata Hexa XTA,₹ 10 Lakh,2179.0,Third Party insurance,Automatic,70000.0,First Owner,Diesel,7,MUV,Agra,17.60,7
2,2,2015 Maruti Swift Dzire VXI,₹ 4.50 Lakh,1197.0,Third Party insurance,Manual,70000.0,Second Owner,Petrol,5,Sedan,Agra,20.85,10
3,3,2015 Maruti Swift Dzire VXI,₹ 4.50 Lakh,1197.0,Third Party insurance,Manual,70000.0,Second Owner,Petrol,5,Sedan,Agra,20.85,10
4,4,2009 Hyundai i10 Magna 1.1,₹ 1.60 Lakh,1086.0,Third Party insurance,Manual,80000.0,First Owner,Petrol,5,Hatchback,Agra,19.81,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17441,17441,2013 Honda Amaze VX i-Vtech,₹ 3.25 Lakh,1198.0,Comprehensive,Manual,89000.0,Second Owner,Petrol,5,Sedan,Delhi,18.00,9
17442,17442,2016 Toyota Camry 2.5 Hybrid,₹ 20.75 Lakh,2494.0,Comprehensive,Automatic,68000.0,First Owner,Petrol,5,Sedan,Delhi,19.16,9
17443,17443,2016 Toyota Corolla Altis GL MT,₹ 8.35 Lakh,1798.0,Comprehensive,Manual,81000.0,First Owner,Petrol,5,Sedan,Delhi,14.28,9
17444,17444,2019 Hyundai Creta 1.6 CRDi AT SX Plus,₹ 13.95 Lakh,1582.0,Comprehensive,Automatic,20000.0,First Owner,Diesel,5,SUV,Delhi,17.01,9


In [24]:
# Check if the column exists and then drop it
if 'Unnamed: 0' in df.columns:
    df.drop(columns=['Unnamed: 0'], inplace=True)
    print("Dropped the 'Unnamed: 0' column.")

Dropped the 'Unnamed: 0' column.


In [25]:
import pandas as pd

# Extract the numeric part by splitting the string and taking the second element
# The .str accessor lets us apply string methods to the whole column
df['price_in_lakhs'] = df['resale_price'].str.split(' ').str[1]

# Convert the new column to a numeric type
df['price_in_lakhs'] = pd.to_numeric(df['price_in_lakhs'], errors='coerce')

# Drop the original 'resale_price' column
df.drop(columns=['resale_price'], inplace=True)

print("Created 'price_in_lakhs' and dropped the old price column.")
print(df[['price_in_lakhs']].head())

Created 'price_in_lakhs' and dropped the old price column.
   price_in_lakhs
0            5.45
1           10.00
2            4.50
3            4.50
4            1.60


In [26]:
# This will show you every unique insurance type in your data
print(df['insurance'].unique())
print(df['owner_type'].unique())

['Third Party insurance' 'Comprehensive' 'Zero Dep' 'Third Party'
 'Not Available' '2' '1']
['First Owner' 'Second Owner' 'Third Owner' 'Fifth Owner' 'Fourth Owner']


In [27]:
# Consolidate 'Third Party' into one category
df['insurance'] = df['insurance'].replace('Third Party', 'Third Party insurance')

# Group '1', '2', and 'Not Available' into a single 'No Insurance' category
df['insurance'] = df['insurance'].replace(['1', '2', 'Not Available'], 'No Insurance')

# Verify the cleaned categories
print("--- Cleaned Unique Values ---")
print(df['insurance'].unique())


--- Cleaned Unique Values ---
['Third Party insurance' 'Comprehensive' 'Zero Dep' 'No Insurance']


In [28]:
import pandas as pd

def extract_brand_and_model(name):
    """
    Intelligently extracts brand and model from a car name string,
    ignoring leading numbers.
    """
    words = name.split()
    brand = None
    model = None
    
    # Find the first word that is not a number (the brand)
    for i, word in enumerate(words):
        if word.isalpha():
            brand = word
            # The next word is assumed to be the model
            if i + 1 < len(words):
                model = words[i+1]
            break
            
    return pd.Series([brand, model])

# Apply the function to your car name column
df[['brand', 'model']] = df['full_name'].apply(extract_brand_and_model)

In [29]:
# Replace 'full_name' with the exact name of your car name column
df = df.drop(columns=['full_name','body_type'])
df


,engine_capacity,insurance,transmission_type,kms_driven,owner_type,fuel_type,seats,city,mileage_numeric,car_age,price_in_lakhs,brand,model
0,1197.0,Third Party insurance,Manual,40000.0,First Owner,Petrol,5,Agra,21.40,8,5.45,Maruti,Baleno
1,2179.0,Third Party insurance,Automatic,70000.0,First Owner,Diesel,7,Agra,17.60,7,10.00,Tata,Hexa
2,1197.0,Third Party insurance,Manual,70000.0,Second Owner,Petrol,5,Agra,20.85,10,4.50,Maruti,Swift
3,1197.0,Third Party insurance,Manual,70000.0,Second Owner,Petrol,5,Agra,20.85,10,4.50,Maruti,Swift
4,1086.0,Third Party insurance,Manual,80000.0,First Owner,Petrol,5,Agra,19.81,16,1.60,Hyundai,i10
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17441,1198.0,Comprehensive,Manual,89000.0,Second Owner,Petrol,5,Delhi,18.00,9,3.25,Honda,Amaze
17442,2494.0,Comprehensive,Automatic,68000.0,First Owner,Petrol,5,Delhi,19.16,9,20.75,Toyota,Camry
17443,1798.0,Comprehensive,Manual,81000.0,First Owner,Petrol,5,Delhi,14.28,9,8.35,Toyota,Corolla
17444,1582.0,Comprehensive,Automatic,20000.0,First Owner,Diesel,5,Delhi,17.01,9,13.95,Hyundai,Creta


In [30]:
df.dropna(subset=['price_in_lakhs'], inplace=True)


df

,engine_capacity,insurance,transmission_type,kms_driven,owner_type,fuel_type,seats,city,mileage_numeric,car_age,price_in_lakhs,brand,model
0,1197.0,Third Party insurance,Manual,40000.0,First Owner,Petrol,5,Agra,21.40,8,5.45,Maruti,Baleno
1,2179.0,Third Party insurance,Automatic,70000.0,First Owner,Diesel,7,Agra,17.60,7,10.00,Tata,Hexa
2,1197.0,Third Party insurance,Manual,70000.0,Second Owner,Petrol,5,Agra,20.85,10,4.50,Maruti,Swift
3,1197.0,Third Party insurance,Manual,70000.0,Second Owner,Petrol,5,Agra,20.85,10,4.50,Maruti,Swift
4,1086.0,Third Party insurance,Manual,80000.0,First Owner,Petrol,5,Agra,19.81,16,1.60,Hyundai,i10
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17441,1198.0,Comprehensive,Manual,89000.0,Second Owner,Petrol,5,Delhi,18.00,9,3.25,Honda,Amaze
17442,2494.0,Comprehensive,Automatic,68000.0,First Owner,Petrol,5,Delhi,19.16,9,20.75,Toyota,Camry
17443,1798.0,Comprehensive,Manual,81000.0,First Owner,Petrol,5,Delhi,14.28,9,8.35,Toyota,Corolla
17444,1582.0,Comprehensive,Automatic,20000.0,First Owner,Diesel,5,Delhi,17.01,9,13.95,Hyundai,Creta


In [31]:
df.to_csv('cleaned_car_data.csv', index=False)

In [32]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

In [33]:
X = df.drop('price_in_lakhs', axis=1)
y = df['price_in_lakhs']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [34]:
owner_categories = ['First Owner', 'Second Owner', 'Third Owner', 'Fourth Owner', 'Fifth Owner']
insurance_categories = ['Comprehensive', 'Zero Dep', 'Third Party insurance', 'No Insurance']

In [35]:
ordinal_cols = ['owner_type', 'insurance']
onehot_cols = [ 'city', 'fuel_type', 'transmission_type','brand','model']
numeric_cols = X.select_dtypes(include=['number']).columns.tolist()
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_cols),
        ('ordinal', OrdinalEncoder(categories=[owner_categories, insurance_categories]), ordinal_cols),
        ('onehot', OneHotEncoder(handle_unknown='ignore', drop='first'), onehot_cols)
    ],
    remainder='passthrough' # Keep other columns (if any), though we've covered all
)


In [36]:
model = RandomForestRegressor(n_estimators=100, random_state=42)


In [37]:
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', model)
])

In [38]:
from sklearn.model_selection import RandomizedSearchCV

# Define the hyperparameter grid to search
param_grid = {
    'regressor__n_estimators': [50, 100, 150, 200, 300, 500, 600, 700, 800, 900, 1000, 1100, 1200],
    'regressor__max_features': ['sqrt'],
    'regressor__max_depth': [int(x) for x in np.linspace(5, 30, num=6)] + [None],
    'regressor__min_samples_split': [2, 5, 10],
    'regressor__min_samples_leaf': [1, 2, 4]
}
# Create the RandomizedSearchCV object
rf_random_search = RandomizedSearchCV(estimator=pipeline, 
                                      param_distributions=param_grid, 
                                      n_iter=100,
                                      cv=5,
                                      verbose=2, 
                                      random_state=42, 
                                      n_jobs=-1,
                                      error_score='raise') # <-- ADD THIS LINE

# Now, run the fit method again
rf_random_search.fit(X_train, y_train)

# Print the best parameters found
print(rf_random_search.best_params_)

Fitting 5 folds for each of 100 candidates, totalling 500 fits
{'regressor__n_estimators': 900, 'regressor__min_samples_split': 2, 'regressor__min_samples_leaf': 1, 'regressor__max_features': 'sqrt', 'regressor__max_depth': None}


In [39]:
from sklearn import metrics

# The best model is automatically stored
final_model = rf_random_search.best_estimator_

# Make predictions on the test data
predictions = final_model.predict(X_test)

# Check the performance
r2_score = metrics.r2_score(y_test, predictions)
mae = metrics.mean_absolute_error(y_test, predictions)

print(f"Final Model R-squared (R²): {r2_score:.2f}")
print(f"Final Model Mean Absolute Error (MAE): ₹{mae:,.0f}")

C:\Users\phala\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_encoders.py:246: UserWarning: Found unknown categories in columns [3, 4] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


Final Model R-squared (R²): 0.83
Final Model Mean Absolute Error (MAE): ₹2


In [40]:
import pickle

# Save the entire pipeline object
with open('car_price_pipeline.pkl', 'wb') as file:
    pickle.dump(final_model, file)

print("\nModel saved as 'car_price_pipeline.pkl'")


Model saved as 'car_price_pipeline.pkl'
